
#**Machine Learning Projects Milestone 2 Report**

\begin{align}
  \text{Alexander Sharpe: sharpeal@bc.edu} \\
  \text{Andrea Brigliadori: brigliad@bc.edu} \\
  \text{Alan Chang: changwg@bc.edu} \\
  \text{Jasroop Dhingra: dhingraj@bc.edu} \\
  \text{Cole Dixon: dixonco@bc.edu}
\end{align}



---

## Missing parts for milestone 2

3.1.2) Explain why the data is good to solve the problem

Explain which member of the group will do each part.

3.5) Explain who has done what between Milestone 1 and Milestone 2 (this is different from 3.4 which asks about who will do what in the future)


## Introduction
Biosensors are becoming an increasingly popular method of detecting specific chemical substances via the combination of a biological component and modern technologies. These sensors can be used in a variety of contexts, one such being the ability to detect proteins in a water source. For example, it may be useful to detect a protein affine to COVID-19, or any other potentially harmful protein, in a source of drinking water. We can employ biosensors in the form of an artificial nucleic acid segment, called an aptamer: each individual protein reacts in a unique way to the aptamer, so certain aptamers can be used on a biosensor to capture specific proteins. The crucial point is the binding affinity between a protein-aptamer pair, which can be measured with kd values, were lower figures indicate better affinity. In a traditional lab setting, testing the binding affinity between one protein and aptamer pair proves too costly in terms of both time and money. This is where machine learning comes in. A database PDBBind containing numerous aptamer-protein combinations and their interactions exists. We plan to use this dataset to train a machine learning model that, given protein-aptamers complexes as input is able to rank them in order of binding affinity. This implicitly helps identify which aptamers bind best to which proteins and may find applications in the search for a simpler and time-saving way to build biosensors able to detect proteins.

## Expectations
In our case good behavior would mean reaching a classifier able to rank aptamer-protein interactions with high enough quality. A positive result could also be a prediction considered unexpected, but that can be justified from a biological perspective.
A negative situation could be that the classifiers we're able to make don't manage to recognize significant patterns in the data or show a tendency to overfit, resulting in a low accuracy. Or, it could happen that we can obtain sufficient results only at the cost of high inefficiency and long running times. Also, obtaining results that we're unable to interpret or explain may represent a complication.

## Related Work:  
"Deciphering interaction fingerprints from protein
molecular surfaces using geometric deep learning" by Gazina et al. uses geometric deep learning models to do three tasks:


1.   Given the surface and chemical properties of a protein binding site, it can predict how well a ligand (small ligand like metabolites) binds to that binding site.
2.   Given the shape and chemical features of the entire protein, it can also predict what the binding site of the protein is. The calculation of the binding site/sites seems to be universal. However, this is still a little unclear.
3.   Given a pair of binding sites/interacting surface patches from different proteins, it can score how well these patches bind to each other.

Although this paper focuses more on protein-protein interactions, we believe their models can be applied to protein-aptamer interactions.



"AptaNet as a deep learning approach for aptamer–protein interaction prediction" by Neda Emami and Reza Ferdousi uses a fully-connected neural network to predict whether the binding affinity of a protein-aptamer pair is above a certain threshold. The problem explored in this paper is much simpler than our problem of correctly ranking the binding affinity of different protein-aptamer complexes. Nonetheless, the features that extract and input into their machine learning model could be of use to us.

"AptaBERT: Predicting aptamer binding interactions" by Morsch et al. attempts to achieve higher accuracy on the problem that AptaNet tackles. It attempts to do so (and seemingly does so based on its empirical data) by using a different neural network model. Before passing its dataset into a fully connected neural network, it passes it through a pre-training model BERT (Bidirectional Encoder Representations from Transformers) developed by Google. Since BERT relies on recurrent neural networks, it encodes each protein-aptamer complex as a sequence of its amino acids and nucleic acids. BERT is perhaps a pre-training model that we can use as well.



## Materials and Methods
The Protein Databank used throughout the study contains files that are populated with geometric and chemical properties of each protein. These characteristics will be extracted by means of the molecular dynamics software GROMACS, which enables to create the needed features for training. They will be used in the realization of the training, test, and validation sets. The performance of our model will be evaluated by comparison with a "ground truth", consisting in the existing experimental results.

Aside from the neural network development, one of the main friction points we will have to study will be which input features determine best how well proteins and aptamers bind. What follows is a comprehensive description of the features in our raw data.



ID:

  - Identifier for each entry in the dataset.

  
PDB code:

- PDB stands for Protein Data Bank, and the PDB code is an alphanumeric identifier assigned to a specific macromolecular structure in the PDB.


Subset:

  - Subgroup or category to which the entry belongs. It could be used for grouping similar interactions. This may be a column we want to delete, as it seems that all its entries are 'general'.


Complex Type:

  - Describes the type of protein-aptamer complex, indicating the nature of the interaction (e.g., binding, enzyme-substrate).


Resolution:

  - Resolution of the experimental data, indicating the level of detail in the three-dimensional structure of the protein-aptamer complex. 'NMR' (Nuclear Magnetic Resonance) is a technique that provides information about the atomic-level structure of molecules in solution. The numeric values in the column instead are crystallographic resolutions, expressed in Ångströms (Å). Crystallography is another common method used to determine protein structures. The numeric value represents the resolution of the structure, with lower values indicating higher resolution and better-defined structures.


Affinity Data:

  - Data on the dissociation constant (Kd) of the protein-aptamer interaction, expressed in nanomolars (nM). The dissociation constant is a measure of how tightly an aptamer binds to a protein. A lower Kd value indicates a stronger binding affinity between the two molecules.

pKd, pKi, pIC50:

  - Measures of binding affinity, expressed as negative logarithms (pKd for dissociation constant, pKi for inhibition constant, pIC50 for half-maximal inhibitory concentration. The 'p' in front of the names stands for 'negative logarithm').

Release Year:

  - Indicates the year when the data or structure was released or published.

Protein Name:

  - The name of the protein involved in the interaction.

Ligand Name:

  - Name of the aptamer in the interaction.

Pubmed ID:

  - PubMed Identifier, a unique number assigned to each PubMed article (publication associated with the data).

Pubchem SID:

  - PubChem Substance Identifier, a unique identifier assigned to chemical substances.

EC number:

  - Enzyme Commission number, a numerical classification for enzymes based on the type of chemical reaction they catalyze.

Reference:

  - Additional information or details about the source of the data.

UniProt AC:

  - UniProt Accession Code, a unique identifier assigned to each protein entry in the UniProt database.

We proceed with the imports of the necessary packages and the definitions of useful functions, followed by a preliminary data preprocessing.

### Imports and useful functions

In [2]:
#!pip install scikit-learn numpy pandas umap-learn matplotlib seaborn scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.7 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=8a7bb3c98dd94b94b327d6736e4d49caa7cbe88f793740f1fda7ad3dcafc022f
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, k_means, SpectralClustering
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, f1_score, recall_score, silhouette_score
from sklearn.pipeline import Pipeline, make_pipeline
import umap
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
state = 42 # Random state to use

In [4]:
# Useful functions

## Data preprocessing

def translate_kd(kd):
    kd.strip()
    kd = kd[3:]
    factor = 0
    if "nM" in kd:
        factor = 1
    elif "pM" in kd:
        factor = .001
    elif "uM" in kd:
        factor = 1000
    kd = kd[:-2]
    try:
        return float(kd) * factor
    except:
        print(kd)
        return None

def output(df):
    df_Final = pd.DataFrame(columns=["PDB code 1","Protein Name 1","Ligand Name 1","Affinity Data 1","kd value (nM) 1","PDB code 2","Protein Name 2","Ligand Name 2","Affinity Data 2","kd value (nM) 2","Output"])
    for _ , row1 in df.iterrows():
        for _ , row2 in df.iterrows():
            if row1["PDB code"] == row2["PDB code"]:
                continue
            output = 1 if row1["kd value (nM)"] < row2["kd value (nM)"] else 0
            temp = row1.to_list() + row2.to_list()
            temp.append(output)
            df_Final.loc[len(df_Final.index)] = temp
    return df_Final

## Preparing the data for classification

def random_indices(data, test_ratio):
    """
    Returns shuffled indices, useful for reordering the rows of the datasets.
    """
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return train_indices, test_indices

def split(x, y, rate = 0.2):
    """
    Split train and test.

    """
    train_ind, test_ind = random_indices(x, rate)
    x_train = x[train_ind]
    x_test = x[test_ind]
    y_train = y[train_ind]
    y_test = y[test_ind]

    return x_train, y_train, x_test, y_test

## Data Preprocessing

### Selecting spherical coordinates

In [9]:
#Data with cartesian coordinates converted to spherical coordinates
conv_data1 = pd.read_csv("Data_Spherical_Coordinates.csv", delimiter="\s+", engine='python', skiprows=5, header=None)
print(conv_data1.shape)
angles = conv_data1.iloc[:,1:3]
angles.head

FileNotFoundError: [Errno 2] No such file or directory: 'Data_Spherical_Coordinates.csv'

In [ ]:
kmeans = KMeans(n_clusters=10, n_init="auto", random_state=0)
km_clusters = kmeans.fit_predict(angles)
km_cluster_centers = kmeans.cluster_centers_
plt.figure(figsize=(8, 6))
plt.scatter(angles.iloc[:, 0], angles.iloc[:, 1], c=km_clusters, cmap='viridis', s=5, alpha=0.5)
plt.scatter(km_cluster_centers[:, 0], km_cluster_centers[:, 1], marker='o', c='red', s=30, label='Cluster centers')

plt.title('KMeans Clustering')

plt.grid(True)
plt.show()
print('Cluster Centers:')
print()
print(km_cluster_centers)

In [ ]:
km_cluster_centers_indeces = []
for center in km_cluster_centers:
    distances = np.linalg.norm(angles - center, axis=1)
    index_closest_point = np.argmin(distances)
    km_cluster_centers_indeces.append(index_closest_point)

km_representative_data = conv_data1.iloc[km_cluster_centers_indeces, :]

In [ ]:
spectral = SpectralClustering(n_clusters=10, assign_labels='cluster_qr', random_state=state, n_neighbors=int(np.sqrt(angles.shape[0])))
s_clusters = spectral.fit_predict(angles)
plt.figure(figsize=(8, 6))
plt.scatter(angles.iloc[:, 0], angles.iloc[:, 1], c=s_clusters, cmap='viridis', s=5, alpha=0.5)

s_cluster_centers = np.zeros((10, 2))
for i in range(10):
    s_cluster_points = angles[s_clusters == i]
    s_cluster_centers[i] = np.mean(s_cluster_points, axis=0)
plt.scatter(s_cluster_centers[:, 0], s_cluster_centers[:, 1], marker='o', c='red', s=30, label='Cluster centers')

plt.title('Spectral Clustering')

plt.grid(True)
plt.show()
print('Cluster Centers:')
print()
print(s_cluster_centers)

In [ ]:
s_cluster_centers_indeces = []
for center in s_cluster_centers:
    distances = np.linalg.norm(angles - center, axis=1)
    index_closest_point = np.argmin(distances)
    s_cluster_centers_indeces.append(index_closest_point)

s_representative_data = conv_data1.iloc[s_cluster_centers_indeces, :]

In [ ]:
#Silhouette score for Kmeans
kmeans_silhouette_score = silhouette_score(angles, km_clusters)

#Silhouette score for Spectral Clustering
spectral_silhouette_score = silhouette_score(angles, s_clusters)

print("Indice silhouette per K-Means:", kmeans_silhouette_score)
print("Indice silhouette per Spectral Clustering:", spectral_silhouette_score)
print()
print("We choose the representative data obtained with KMeans:")
km_representative_data

The goal of our data preprocessing was to convert the data into a usable format for training models. The first step was to create our labels based off of the experimental kd values that we have. There are 53 different protein-aptamer complexes that we are looking at, and in order to create labels we need to look at every pair of complexes and determine which one has a lower kd value which is associated with a stronger bond. By framing the problem as a comparison, our number of data points becomes 53*52 = 2757 data points.

In [ ]:
df = pd.read_csv("KD_data.csv")
df.drop(index = 0,inplace=True)
df = df[['PDB code','Protein Name','Ligand Name','Affinity Data']]
df["kd value (nM)"] = [translate_kd(kd) for kd in df["Affinity Data"]]
df_compare = pd.read_csv("Aptamer-Protein-Information.csv")
df = df[df["PDB code"].isin(df_compare["Unnamed: 0"])]
df = output(df)
df.to_csv("data_output.csv",index=False)

### Exploratory Data Analysis

### Feature selection

### Dimensionality Reduction

### Clustering

**ALEX NOTE: INCLUDE IMAGES FOR THE TWO ARCHITECURES**

### Classifiers

One of the things that we tasked ourselves with in Milestone 1 was "Model Selection." In our efforts to find which model's might work best, we did some testing on a couple models. I (Alex S.) have a lot more experience with neural networks than other types of models, so I decided that I would do some research and testing on neural networks for our project. Neural networks are often very good at detecting complex patterns and non-linear relationships, which could potentially be beneficial with our current dataset, as many PA Complexes have similar feature values.

We have currently determined two architectures for our model. The first is one in which the "ML box" is doing classification, in which it outputs the probability that 1 PA Complex has a higher kd value than another, repeating this process until we have a ranking for all the PA Complexes in our dataset. The other is where the "ML Box" does regression, in which it directly outputs the kd values of each PA Complex inputted and then ranks them from least to greatest. Both of these architectures will allow us to accomplish our task, however they each have their own upsides and downsides. By doing classification, this would allow us to have significantly more data to train on, as we would input every possible pair of PA Complexes and their data points into the ML box, whereas for regression we would only input 1 PA Complex at a time into the ML Box. However, if the input features are good enough, the model may be extremely accurate in predicting the exact kd value, thus allowing our ranking to be very accurate as well. The margin of error for the classifcation task may possibly be much larger. For this round of model testing, we tested the second architecure, as shown in the code below. (It is also important to note that we are not doing the actual ranking here, we are just training a model to output the kd value of a PA Complex. To rank them we would just use a sorting algorithm on all the predicted kd values of the test set)

In order to "test" neural networks for our project, I created a dataset that mimics our own: it consists of 100 PA Complexes, each with a random kd value between 0 and 10, and 20 Properties with a random value between 0 and 100. The Properties resemble our input features. Obviously the actual data will not just be random floats, but this dataset will still help us to see if a neural network will work at all on our actual data. In fact, it will be much easier for a model to learn on non-random data where there are clear patterns in the input features that determine the kd value.

The first model that I tested was a simple fully connected neural network with 3 hidden layers. The code and its explanation is attached below.

Step 1: Import Necessary Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

Step 2: Create Environment and Set Hyperparameters

- Hyperparams: learning rate, number of epochs, loss function, batch

In [ ]:
#setting basic environment
torch.manual_seed(100)

#if we have a gpu then run network on that, if not use cpu which is much slower
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#" Varying learning rate between 0.0001 and 0.01 is considered optimal in most of the cases" - https://www.kdnuggets.com/2022/12/tuning-adam-optimizer-parameters-pytorch.html
lr = 0.001
batch_size = 1
num_epochs = 801
loss_fn = nn.L1Loss()

Step 3: Create Data

- Create a Dataframe that mimics our actual data in that there are 10 "PA Complexes," each of which has a numerical value for 20 different "Properties."

- Create a custom dataset for our dataframe, this will allow us to use Pytorch's DataLoader which helps us train

- Split data into test, train, val splits

- Input our data into the DataLoader

In [ ]:
rng = np.random.default_rng(48598585)
data = {}
data.update({"KD Value": [rng.random()*10 for x in range(1,101)]})

for i in range(1, 21):
 data.update({"Property " + str(i): [rng.random()*100 for x in range(1,101)]})

df = pd.DataFrame(data , index = ["PA Complex " + str(x) for x in range(1,101)])

class CustomDataset(Dataset):
    def __init__(self, dataframe):
       self.dataframe = dataframe

    def __getitem__(self, index):
       row = self.dataframe.iloc[index].to_numpy()
       features = row[1:]
       label = row[0]
       return features, label

    def __len__(self):
       return len(self.dataframe)

train_set = df.iloc[0:51]
test_set = df.iloc[51:78]
val_set = df.iloc[78:101]
data1 = CustomDataset(dataframe=train_set)
data2 = CustomDataset(dataframe=test_set)
data3 = CustomDataset(dataframe=val_set)
train_loader = DataLoader(data1, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data2, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(data3, batch_size=batch_size, shuffle=True)

#UNCOMMENT to see what the sample data looks like
# print(test_set)
# for sample in train_loader:
#    print(sample)

Step 4: Create Our Model

- Create Model class

    - def _init__ is the function where we define our model, def forward is where we define our forward pass
    
    - Use ReLU to only pass positive values through the net, thus activating less neurons during each pass, thus allowing for easier computation. Use Leaky form to avoid dying ReLU problem (sometimes neurons may turn off permanently)
    
    - Dropout is used after each layer to reduce overfitting: co-adaptation occurs when multiple neurons extract the same
      or very similar features, thus emphasizing those features in the result. This can lead to over fitting if those
      features are specfic only to the training set. So, dropout randomly shuts down a # of neurons in a layer by
      setting their weights to 0. Frac of neurons zeroed out is **r_d**, all other neurons multiplied by **1/1-r_d** so overall sum of
      neurons stays the same

- Instantiate the object

- Define the optimizer for use in training

In [ ]:
class Simple_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(20, 50),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(50, 100),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.Linear(100, 200),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(200, 50),
            nn.LeakyReLU(),
            nn.Linear(50, 1),
        )

    def forward(self, x):
        output = self.model(x)
        return output

#to run using GPU, we have instantiate as usual but then send to GPU w/ .to() the device
simple_NN = Simple_NN().to(device=device)
optimizer_SNN = torch.optim.Adam(simple_NN.parameters(), lr=lr)

Step 5: Training Process
- We first create an accuracy function that returns the percentage accuracy, i.e. if the predicted kd is 3, and the true kd is 4, then predicted value is 75% of what we want

- Next define train function
    - We include 3 seperate blocks for each dataset. The model's weights are updated by training on the train set, and the model's hyperparameters are updated by checking the performance on the validation set for each epoch. We then exit the training loop (when we have gone through every epoch) and evaluate on the test set

    - We create 3 graphs, one for the training loss, one for the test loss on each PA Complex (in the test set), and one for the test accuracy on each PA Complex

    - We employ a number of functions and methods to train the model:
        - Adam Optimizer: Updates weights with optimizer.step()
        - Learning Rate Scheduler: decreases the learning rate every 100 epochs to avoid overshooting the minimum via scheduler.step()
        - L1 Loss = Mean Absolute Error Loss: Updated via loss.backward()
        - Use of model.train() and model.eval() b/c certain things like dropout layers funtion differently in Pytorch depending on if you want to train or evalulate

In [ ]:
#UNCOMMENT to see untrained model predictions on train set
# for i, data in enumerate(test_loader):
#     values, labels = data
#     values = values.to(torch.float32)
#     untrained_preds = simple_NN(values)
#     print(untrained_preds)
#     print(labels)

def accuracy_fn(y_true, y_pred):
    return y_pred.item() / y_true.item()

def train(model, num_epochs, loss_fn, train_data, val_data, test_data, optimizer):
    epoch_loss1 = []
    epoch_loss2 = []
    train_acc = []
    val_acc = []
    test_acc =[]
    train_losses = []
    val_losses = []
    test_losses = []
    scheduler = lr_scheduler.StepLR(optimizer, step_size = 100, gamma=0.9)

    for epoch in range(num_epochs):

        model.train()
        for i, data in enumerate(train_data):
            values, label = data
            values = values.to(torch.float32)
            label = label.to(torch.float32)
            optimizer.zero_grad()
            train_preds = model(values)
            train_loss = loss_fn(train_preds, label)
            epoch_loss1.append(train_loss.item())
            train_acc.append(accuracy_fn(label, train_preds))
            train_loss.backward()
            optimizer.step()
            scheduler.step()
        train_losses.append(np.mean(epoch_loss1))

        model.eval()
        for j, data in enumerate(val_data):
            values, label = data
            values = values.to(torch.float32)
            label = label.to(torch.float32)
            val_preds = model(values)
            val_loss = loss_fn(val_preds, label)
            epoch_loss2.append(val_loss.item())
            val_acc.append(accuracy_fn(label, val_preds))
        val_losses.append(np.mean(epoch_loss2))

        if epoch % 20 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Train Loss: {np.mean(epoch_loss1):.4f}, Train Accuracy: {np.mean(train_acc):.2f}, Val Loss: {np.mean(val_losses):.4f} Val Accuracy: {np.mean(val_acc):.2f}")

    for k, data in enumerate(test_data):
        values, label = data
        values = values.to(torch.float32)
        label = label.to(torch.float32)
        test_preds = model(values)
        test_loss = loss_fn(test_preds, label)
        test_losses.append(test_loss.item())
        test_acc.append(accuracy_fn(label, test_preds))

    plt.plot(np.arange(num_epochs), train_losses)
    plt.title("Training Loss Graph")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(np.arange(27), test_losses)
    plt.title("Test Loss for Each PA Complex")
    plt.xlabel("PA Complex")
    plt.ylabel("Loss")
    plt.show()

    plt.bar(np.arange(27), test_acc)
    plt.title("Accuracy for Each PA Complex")
    plt.xlabel("PA Complex")
    plt.ylabel("Accuracy")
    plt.show()

train(simple_NN, num_epochs, loss_fn, train_loader, val_loader, test_loader, optimizer_SNN)

Step 6: Model Analysis

**TO COMPLETE**

### Conclusions

### References

# Data Preprocessing (Cole)
The goal of our data preprocessing was to convert the data into a usable format for training models. The first step was to create our labels based off of the experimental kd values that we have. There are 53 different protein-aptamer complexes that we are looking at, and in order to create labels we need to look at every pair of complexes and determine which one has a lower kd value which is associated with a stronger bond. By framing the problem as a comparison, our number of data points becomes 53*52 = 2757 data points.

In [ ]:
'''import pandas as pd
import numpy as np'''

In [5]:
'''def translate_kd(kd):
    kd.strip()
    kd = kd[3:]
    factor = 0
    if "nM" in kd:
        factor = 1
    elif "pM" in kd:
        factor = .001
    elif "uM" in kd:
        factor = 1000
    kd = kd[:-2]
    try:
        return float(kd) * factor
    except:
        print(kd)
        return None

def output(df):
    df_Final = pd.DataFrame(columns=["PDB code 1","Protein Name 1","Ligand Name 1","Affinity Data 1","kd value (nM) 1","PDB code 2","Protein Name 2","Ligand Name 2","Affinity Data 2","kd value (nM) 2","Output"])
    for _ , row1 in df.iterrows():
        for _ , row2 in df.iterrows():
            if row1["PDB code"] == row2["PDB code"]:
                continue
            output = 1 if row1["kd value (nM)"] < row2["kd value (nM)"] else 0
            temp = row1.to_list() + row2.to_list()
            temp.append(output)
            df_Final.loc[len(df_Final.index)] = temp
    return df_Final'''

In [6]:
'''df = pd.read_csv("KD_data.csv")
df.drop(index = 0,inplace=True)
df = df[['PDB code','Protein Name','Ligand Name','Affinity Data']]
df["kd value (nM)"] = [translate_kd(kd) for kd in df["Affinity Data"]]
df_compare = pd.read_csv("Aptamer-Protein-Information.csv")
df = df[df["PDB code"].isin(df_compare["Unnamed: 0"])]
df = output(df)
df.to_csv("data_output.csv",index=False) '''

FileNotFoundError: [Errno 2] No such file or directory: 'KD_data.csv'

# Current Member Contributions:

Alex: **Milestone 1]** Took notes on potential projects during thr first meeting. Created and structured the Colab Notebook. Wrote the "Problem" section based off of the group's current understanding of the problem we hope to solve. Wrote the Task Organization section with Alan.

Jasroop: **Milestone 1]** Participated in discussions for potential project topics. Created Github repo for the project. Read research papers to understand background knowledge for the project.

Cole: **Milestone 1]** Contributed to discussion about project ideas. Wrote 'Model Selection Approach' section.

Alan: **Milestone 1]** Added to and edited "Problem" section. Wrote the "Similar Problems" Section. Begun writing and executing python code for feature extraction using GROMACS.

Andrea:
  - **Milestone 1]**  I participated in and contributed to the meetings by proposing ideas for a project, suggesting ways to conduct feature selection using ANOVA or other techniques to assess statistical significance, and providing sources useful to have a better understanding of the problem and the PDB dataset. I wrote the paragraphs 'Examples', 'Report and Code Organization' and 'Data description', to better organize our objectives for the next phase of the project.
  - **(Milestone 1, Milestone 2]** I contributed to the meetings and to the discussions on how to proceed with the project. I converted the cartesian coordinates relative to the protein-aptamer bonds into spherical coordinates. Then I applied KMeans and Spectral clustering, found ten cluster centers and compared the quality of the results using silhouette score. This indicated as preferable the results obtained with KMeans, and the coordinates of the corresponding cluster centers will be used as the most representative angles at which the protein-aptamer bonds form. I reorganized and corrected part of the report from Milestone 1.

# Future Tasks Organization:

These are the current tasks that we will be working on in the near future. We expect to encounter more tasks as we go along, but this is just what we are dealing with at the moment.

Choose Input Features: Alan, Andrea

  - Requires physics and chemistry knowledge
  - Determine which input features to use in order to accurately find the best protein + aptamer combination

Data Preprocessing: Jasroop

  - Transfer data into matrix/vector
  - Will most likely need to be familar with numpy, SKLearn, and others

Model Selection: Alex, Cole

  - Two people
  - Find which models will be best for our task
  - Familiarize with the APIs (Tensorflow? Pytorch? Others?)

Data Analysis: Cole

  - After the model is chosen, first compare with a random model that randomly matches proteins to aptamers; our model should at the very least beat the random one to start
  - Then go on to compare with other models
